## Working with Related Objects

In this section, we will cover one more _essential ORM concept_, which is __how the ORM interacts with mapped classes that refer to other objects__. In the section `Declaring Mapped Classes`, the _mapped class_ examples made use of a construct called `relationship()`. This construct __defines a linkage__ between _two different mapped classes_, or from a _mapped class to itself_, the _latter_ of which is called a __`self-referential relationship`__. To describe the _basic idea_ of `relationship()`, first we'll review the mapping in short form, omitting the _Column mappings_ and other directives.

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [3]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [5]:
mapped_registry.metadata.create_all(engine)

2022-10-09 11:54:08,693 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-09 11:54:08,695 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-09 11:54:08,696 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-09 11:54:08,698 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-09 11:54:08,699 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-09 11:54:08,700 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-09 11:54:08,702 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-09 11:54:08,704 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-09 11:54:08,705 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-09 11:54:08,707 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-09 11:54:08,709 INFO sqlalchemy.engine.Engine [no key 0.00161s] ()
2022-10-09 11:54:08,713 INFO sqlalchemy.engine.Engine 
C

Above, the `User` class now has an _attribute_ `User.addresses` and the `Address` class has an _attribute_ `Address.user`. The `relationship()` construct will be used to __inspect the table relationships__ between the `Table` objects that are __mapped__ to the `User` and `Address` classes. As the `Table` object representing the _address table_ has a `ForeignKeyConstraint` which refers to the *user_account table*, the `relationship()` can _determine_ __unambiguously__ that there is a __one to many relationship__ _from_ `User.addresses` _to_ `User`; _one particular row_ in the `user_account` table __may be referred towards__ by _many rows_ in the `address` table.

_All one-to-many relationships_ __naturally correspond__ to a _many to one relationship_ in the _other direction_, in this case the one noted by `Address.user`. The `relationship.back_populates` parameter, seen above configured on both `relationship()` objects __referring__ to the _other name_, establishes that each of these two `relationship()` constructs should be __considered to be complimentary__ to _each other_; we will see how this plays out in the next section.

#### Persisting and Loading Relationships

We can start by _illustrating_ what `relationship()` does to instances of objects. If we make a new `User` object, we can note that there is a _Python list_ when we access the `.addresses` element.

In [6]:
u1 = User(name="pkrabs", fullname="Pearl Krabs")
print(f"{u1.addresses = }")

u1.addresses = []


This object is a _SQLAlchemy-specific version_ of _Python list_ which has the __ability to track and respond to changes__ made to it. The _collection_ also appeared __automatically__ when we _accessed the attribute_, even though we __never assigned__ it to the object. This is similar to the behavior noted at `Inserting Rows with the ORM` where it was observed that _column-based attributes_ to which we _don't explicitly assign a value_ also display as `None` __automatically__, rather than raising an `AttributeError` as would be Python's usual behavior.

As the `u1` object is still __transient__ and the _list_ that we got from `u1.addresses` has __not been mutated__ (i.e. _appended or extended_), it's __not actually `associated` with the object__ yet, but as we _make changes_ to it, it will become _part of the state_ of the `User` object.

The _collection_ is specific to the `Address` class which is the _only type of Python object_ that may be __persisted within it__. Using the `list.append()` method we may __add an Address object__.

In [8]:
a1 = Address(email_address="pearl.krabs@gmail.com")
u1.addresses.append(a1)

At this point, the `u1.addresses` _collection_ as expected _contains_ the __new `Address` object__.

In [9]:
print(f"{u1.addresses = }")

u1.addresses = [Address(id=None, email_address='pearl.krabs@gmail.com')]


As we _associated_ the `Address` object with the `User.addresses` _collection_ of the `u1` instance, another behavior also occurred, which is that the `User.addresses` _relationship_ __synchronized itself__ with the `Address.user` _relationship_, such that we __can navigate__ not only from the _User object to the Address object_, we __can also navigate__ from the _Address object back to the "parent" User object_.

In [10]:
print(f"{a1.user = }")

a1.user = User(id=None, name='pkrabs', fullname='Pearl Krabs')


This _synchronization_ occurred as a result of our _use of the_ `relationship.back_populates` parameter between the two `relationship()` objects. This parameter __names__ _another_ `relationship()` for which _complementary attribute_ __assignment/list mutation__ should occur. It will __work equally well__ in the _other direction_, which is that if we _create another_ `Address` _object_ and assign to its `Address.user` attribute, that `Address` becomes part of the `User.addresses` _collection_ on that `User` object.

In [11]:
a2 = Address(email_address="pearl@aol.com", user=u1)
print(f"{u1.addresses = }")

u1.addresses = [Address(id=None, email_address='pearl.krabs@gmail.com'), Address(id=None, email_address='pearl@aol.com')]


We actually made use of the `user` parameter as a _keyword argument_ in the `Address` constructor, which is __accepted just like any other mapped attribute__ that was _declared_ on the `Address` class. It is __equivalent__ to _assignment_ of the `Address.user` attribute after the fact.

In [12]:
# equivalent effect as a2 = Address(user=u1)
a2.user = u1